<a href="https://colab.research.google.com/github/bnriiitb/whisper-fine-tuning/blob/main/model_evaluation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec 24 00:59:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:34:00.0 Off |                    0 |
| N/A   45C    P0   366W / 350W |  26514MiB / 32768MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:36:00.0 Off |                    0 |
| N/A   

In [11]:
import numpy as np
import pandas as pd
import os
from IPython import display
import datasets
from datasets.info import DatasetInfosDict
from datasets import load_dataset,concatenate_datasets, DatasetDict, Dataset, Audio

In [12]:
BASE_PATH = "/raid/cs20mds14030/telugu_asr/data"
def create_dataset_from_metadata(dataset_name):
    ds = load_dataset('csv', data_files=f"{BASE_PATH}/{dataset_name}/metadata.csv")
    # add the complete file name
    ds = ds.map(lambda x: {'file_name':BASE_PATH+"/"+dataset_name+"/"+x["file_name"]})
    return ds['train']

In [13]:
mucs_dataset = create_dataset_from_metadata("mucs/te-in-Test/Audios")
mucs_dataset

Using custom data configuration default-6f72d14d2ba0e3c7
Found cached dataset csv (/raid/cs20mds14030/.cache/huggingface/datasets/csv/default-6f72d14d2ba0e3c7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 507.97it/s]
Loading cached processed dataset at /raid/cs20mds14030/.cache/huggingface/datasets/csv/default-6f72d14d2ba0e3c7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e99aa3274dba06a2.arrow


Dataset({
    features: ['file_name', 'transcription', 'duration'],
    num_rows: 3040
})

In [14]:
# normalize the sampling rate to 16k Hz
mucs_dataset = mucs_dataset.cast_column("file_name",Audio(sampling_rate=16000))

# rename the column file_name to audio
mucs_dataset = mucs_dataset.rename_column("file_name", "audio")

audio_duration = round((np.sum(mucs_dataset['duration'])/60)/60)
print({f"The dataset has {mucs_dataset.num_rows} rows worth {audio_duration} hours of data"})

{'The dataset has 3040 rows worth 5 hours of data'}


In [15]:
mucs_dataset = mucs_dataset.remove_columns(['duration'])
mucs_dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 3040
})

In [16]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import soundfile as sf
import torch
import evaluate
metric = evaluate.load("wer")

In [17]:
mucs_dataset['transcription'][:2]

['ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలను వారికి వివరించారు',
 'ఓహియో అయోవా పెన్సిల్వేనియా రాష్ట్రాల్లో ఒబామా అధిక్యం కొనసాగుతోంది']

In [18]:
import time
from tqdm import tqdm
from transformers import pipeline
pipe = pipeline(model="bnriiitb/whisper-small-te-146h",language='Telugu')

In [19]:
device = "cuda:15" if torch.cuda.is_available() else "cpu"

In [20]:
pipe(mucs_dataset[0]['audio']['array'],language='Telugu')

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': 'ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలు వారికి వివరించారు'}

In [38]:
finetuned_texts = []

for item in tqdm(mucs_dataset['audio']):
    finetuned_texts.append(pipe(item['array'],language='Telugu')['text'])

finetuned_texts

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3040/3040 [4:07:30<00:00,  4.88s/it]


['ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలు వారికి వివరించారు',
 'ఓహియో అయోవా పెన్సిలేనియా రాష్ట్రాల్లో ఒబామా ఆధిక్యం కొనసాగుతోంది',
 'చంద్రబాబు అందరికీ క్షమాపణ చెప్పాలని ఆయన రాజీనామా చేసే వరకు వదలకూడదన్నారు',
 'ఆస్పత్రిలో సంభాశివుడు మృతి చెందగా బాలిక చికిత్స పొందుతుంది',
 'పెద్ద నుడ్ల రద్దు కారణంగా రాష్ట్ర ఖజానాకు గండి పడడంతో',
 'రెండుల కిందట ఇంటి నుంచి పారిపై వివాహం చేసుకున్నారు',
 'ఈ అంతరిక్ష వస్తువులు అంతకు ఆరు రెట్లు ఎక్కువ వేగంతో వెళ్తున్నాయి',
 'పుష్కారాలలో యొక్క నాటికి ఏక పనులు పూర్తి అయిక పరిస్థితిని కనిపించడం లేదు',
 'పిల్లపాపులతో నూరుల జీవనం సాగించాల్సి ఉన్న తన కుమారుడు ఇలా మృతిచడంతో తల్లి పద్మవాతి కుండలు కదిలేలా రోదించడంతో చూపరులు కలిచివేసింది',
 'కాదా ఘటనలో మృతి చెందిన వారికి భారత్తో పాటు పలు దేశాల ప్రజాప్రతి నిధులు తమ ప్రగాఢ సానుభూతిని తెలియజేశారు',
 'పతిపాటి జగన్ వాదనలను ప్రజలు చూస్తారని ఆరోపణలు చేసి వెళ్లిపోవడం సరికాదన్నారు',
 'రోజా చేసిన వ్యాఖ్యల విషయంలోగానే ఆమె అనుసరించిన తీరుపై గాని జగన్ వ్యూహం మరోవిధంగా ఉండాల్సింది',
 'మంత్రులు అధికారపాటి ఎమ్మెల్యేలు 

In [39]:
finetuned_wer = 100 * metric.compute(predictions=finetuned_texts, references=mucs_dataset['transcription'])
finetuned_wer

35.35705486924999

In [110]:
import requests
import json

import base64
import ast

def wav_to_base64string(file_path):
    return base64.b64encode(open(file_path, "rb").read()).decode('utf-8')

url = "https://ai4b-dev-asr.ulcacontrib.org/asr/v1/recognize/te"

def get_payload(base64_audio_content):
    payload = json.dumps({
      "config": {
        "language": {
          "sourceLanguage": "te"
        },
        "transcriptionFormat": {
          "value": "transcript"
        },
        "audioFormat": "wav"
      },
      "audio": [
        {
          "audioContent": f"{base64_audio_content}"
        }
      ]
    })
    return payload

headers = {
  'Content-Type': 'application/json'
}

In [99]:
# wav_to_base64string("/raid/cs20mds14030/telugu_asr/data/mucs/te-in-Test/Audios/001050238.wav")

In [80]:
# base64.b64encode(open("/raid/cs20mds14030/telugu_asr/data/mucs/te-in-Test/Audios/001050238.wav").read())

In [107]:
for i in mucs_dataset['audio'][:2]:
    print(i['path'])

/raid/cs20mds14030/telugu_asr/data/mucs/te-in-Test/Audios/001050238.wav
/raid/cs20mds14030/telugu_asr/data/mucs/te-in-Test/Audios/002100128.wav


In [117]:
indicwav2vec_texts = []
for file in tqdm(mucs_dataset['audio']):
    encoded_string = wav_to_base64string(file['path'])
    response = requests.request("POST", url, headers=headers, data=get_payload(encoded_string))
    indicwav2vec_texts.append(ast.literal_eval(response.text)['output'][0]['source'])
indicwav2vec_texts    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3040/3040 [38:57<00:00,  1.30it/s]


['ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలను వారికి వివరించారు',
 'ఓహియో అయోవా పెన్సిల్వేనియా రాష్ట్రాల్లో ఒబామా అధిక్యం కొనసాగుతోంది',
 'చంద్రబాబు అందరికీ క్షమాపణ చెప్పాలని ఆయన రాజీనామా చేసే వరకు వదలకూడదన్నారు',
 'ఉత్పత్తిలో సాంబశివుడు మృతి చెందగా బాలిక చికిత్స పొందుతోంది',
 'పెద్ద నోట్ల రద్దు కారణంగా రాష్ట్ర ఖజానాకు గండి పడడంతో',
 'రెండేళ్ల కిందట ఇంటి నుంచి పారి వివాహం చేసుకున్నారు',
 'ఈ అంతరిక్ష వస్తువులు అంతకు ఆరు రెట్లు ఎక్కువ వేగంతో వెళ్తున్నాయి',
 'పుష్కరాల   నాటికి పనులు పూర్తయ్యే పరిస్థితి కనిపించడం లేదు',
 'పిల్లపాపలతో నూరు జీవనం సాగించాల్సి ఉన్న తన కుమారుడు ఇలా మృతి చడంతో తల్లి పద్మావతి గుండెలు కదిలేలా రోదించడంతో చూపరుల కలిచివేసింది',
 'కాగా ఘటనలో మృతి చెందిన వారికి భారత్ తోపాటు పలు దేశాల ప్రజా ప్రతినిధులు తమ ప్రగాఢ సానుభూతిని తెలియజేశారు',
 'పత్తిపాటి జగన్ వాదనలను ప్రజలు చూశారని ఆరోపణలు చేసి వెళ్లిపోవడం సరికాదన్నారు',
 'రోజా చేసిన వ్యాఖ్యల విషయంలో గానీ ఆమె అనుసరించిన తీరుపై గానీ జగన్ వ్యూహం మరో విధంగా ఉండాల్సింది',
 'మంత్రులు అధికార పార్టీ ఎమ్మెల్యేలు నేతల ను

In [114]:
mucs_dataset['transcription'][:2]

['ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలను వారికి వివరించారు',
 'ఓహియో అయోవా పెన్సిల్వేనియా రాష్ట్రాల్లో ఒబామా అధిక్యం కొనసాగుతోంది']

In [118]:
indicwav2vec_wer = 100 * metric.compute(predictions=indicwav2vec_texts, references=mucs_dataset['transcription'])
indicwav2vec_wer

15.23246401295182

In [116]:
indicwav2vec_texts

['ఈ సందర్భంగా చంద్రబాబు అమరావతి నగర అభివృద్ధిపై తన ఆలోచనలను వారికి వివరించారు',
 'ఓహియో అయోవా పెన్సిల్వేనియా రాష్ట్రాల్లో ఒబామా అధిక్యం కొనసాగుతోంది']